In [1]:
var ds = require('dstools');
var input = require('../examples/data/3features/input-m.json'); 
var kreativesFeld = require('../index');

#### Wie schneiden die beiden Algorithmen bei 100x Tests ab?  

Gruppengröße: 26  
Gruppenanzahl: 3

In [2]:
var groups = 3;

In [3]:
// 100x runs of monte carlo
var scoresMonteCarlo = [];
var resultsMonteCarlo = {};

// Initialize empty promise collection
var runs = [];

for(let run = 0; run < 100; run++) {
    
    // Calculate groups with monte carlo
    runs.push(kreativesFeld.monteCarlo(input, groups).then(result => {
                
        // Collect result and score
        resultsMonteCarlo[result.score] = result;
        scoresMonteCarlo.push(result.score);
        
        return;
    }));
}

var highscoreMonteCarlo = undefined;

// Run async tasks in a sequence
runs.reduce((promiseChain, currentTask) => {    
    return promiseChain.then(chainResults =>        
             currentTask.then(currentResult =>            
                  [ ...chainResults, currentResult ]        
             )    
        );
    },Promise.resolve([])).then(arrayOfResults => {    

        // Do something with all results
        console.log(scoresMonteCarlo);

        // Calculate highest score of this batch
        highscoreMonteCarlo = Math.max(...scoresMonteCarlo);

        console.log(highscoreMonteCarlo);
    });

[
  223.82383248580317, 223.11081212405517, 233.03395421382254, 227.03950286715343,
   221.5397595232798, 223.29777003879605, 241.18566341964626, 230.57562580954408,
   223.6613470239592, 214.57026081051063, 211.53713942763764,  221.7720685529746,
  213.43863345289174, 215.38806897218657,  224.6163640911817,  221.8798001380449,
  217.75772948582667, 226.40444855129576, 219.83846234502704,  219.8779249069148,
  207.32987296250735,  206.7539959488565, 211.75586776121943, 221.74615151155737,
  207.59317566012425, 223.61371629563524,  223.0416585798933, 227.84864962361925,
  223.44256791230228, 226.47182272008587, 209.11649344797155, 219.12735940516222,
  222.31131655143176, 225.70095333958966,  222.1079253581245, 218.44277999447442,
  239.11339324146806, 216.29390302591833,  224.6351707394622, 218.05579719246708,
  230.25788026912645, 222.61549229331652,  228.7776113236826, 223.42185641150118,
  221.25785097077227, 218.02963771466588,   204.583079883606,  222.4516690144874,
  218.80315434

In [4]:
// 100x runs of genetic
var scoresGenetic = [];
var resultsGenetic = {};
var runs = [];

// Run algorithm x times
for(let run = 0; run < 100; run++) {
    runs.push(kreativesFeld.genetic(input, groups).then(result => {
        
        // Collect result and score
        resultsGenetic[result.score] = result;
        return scoresGenetic.push(result.score);
    }));
}

var highscoreGenetic = undefined;

// We must wait until all calculations are done
Promise.all(runs).then(values => {
    
    console.log(scoresGenetic);

    // Calculate highest score of this batch
    highscoreGenetic = Math.max(...scoresGenetic);

    console.log(highscoreGenetic);
});

[
  224.37477802131318,  196.0384930178766, 214.51150200267915, 216.51048032231884,
    200.241102996707, 222.09728401190551,  234.5059535955467, 216.94394185777608,
   216.4838850313254,  234.1265911775872,  216.3081852708379,  212.7914360331232,
  202.54411660865443, 204.11368475045907,  226.0391090630756,  220.3062878856195,
  227.34380674153599, 227.58126285168666, 226.68952795001138,  234.8151955103284,
  220.51548870311277, 217.51970038313894, 210.99721759729795, 214.86944980767626,
    205.522149589334, 218.84726148929397,  212.6251253996082, 193.30398140647006,
  223.41356058201586, 204.03021758105646, 198.63971664790478,  220.8565351429036,
  203.20721600242717, 230.01929728682902, 210.76988810050483,  206.1089710954402,
  218.95070547710603, 228.93416304730772, 199.12657499808657, 230.46482211898092,
   219.6504847731949,  198.1320326363681, 210.64456837998245,  199.3664806610932,
  208.25960002506358, 217.56649814190985, 215.77277728794388, 216.62550837209142,
   218.1340598

## Vergleich Algorithmen

In [5]:
// Calculate overall hitrate

// Highest score
var highestScore = (highscoreMonteCarlo > highscoreGenetic) ? highscoreMonteCarlo : highscoreGenetic;

var hitRateMonteCarlo = scoresMonteCarlo.map(v => v/highestScore);
var hitRateGenetic = scoresGenetic.map(v => v/highestScore);

In [10]:
console.log(ds);

{
  Collection: [Function (anonymous)] { registerFunction: [Function (anonymous)] },
  HTML: [Function: HTML],
  Wrapper: [Function (anonymous)] { registerFunction: [Function (anonymous)] },
  NOWRAP: [Function: NOWRAP],
  ProgressReporter: [Function (anonymous)]
}


In [11]:
var trace1 = {
  x: hitRateMonteCarlo,
  type: "histogram",
  opacity: 0.5,
  marker: {
     color: 'green',
  },
  name: 'Monte Carlo'
};
var trace2 = {
  x: hitRateGenetic,
  type: "histogram",
  opacity: 0.6,
  marker: {
     color: 'red',
  },
  name: 'Genetic Algorithm'
};

var data = [trace1, trace2];
var layout = {barmode: "overlay", xaxis: {title: 'Trefferquote'}, yaxis: {title: 'Häufigkeit'}};

ds.Collection().plotly(data, layout).show();

## Darstellung "bestes" Ergebnis

In [12]:
// Retrieve winner combination
var winnerSet = (highestScore == highscoreMonteCarlo) ? resultsMonteCarlo : resultsGenetic;
var winner = winnerSet[highestScore];

// Generate empty traces
/*var traces = ['rgba(217, 217, 217, 0.14)', 'rgba(217, 217, 217, 0.14)', 'rgba(217, 217, 217, 0.14)'].map(c => {
    return {x: [], y: [], z: [],
        mode: 'markers',
        marker: {
            size: 12,
            line: {
            color: c,
            width: 0.5},
            opacity: 0.8},
        type: 'scatter3d'};
});
*/
var traces = [];
for(let i = 0; i < groups; i++) {
    traces.push({x: [], y: [], z: [],
        mode: 'markers',
        marker: {
            size: 12,
            line: {
                color: 'rgba(217, 217, 217, 0.14)',
                width: 0.5
            },
            opacity: 0.8
        },
        type: 'scatter3d'
    });
}

// Loop input data and build group traces
Object.keys(input).forEach((member, index) => {
    
    // Retrieve member group 
    var group = winner.combination[index];
    
    traces[group-1].x.push(input[member][0]);
    traces[group-1].y.push(input[member][1]);
    traces[group-1].z.push(input[member][2]);    
});

// Print 3D Model with group traces and colors
ds.Collection().plotly(traces,{margin: {l: 0, r: 0, b: 0, t: 0 }}).show();